In [65]:
import math

import numpy as np
import tensorflow as tf

In [2]:
text = ['he is the king', 'the king is royal', 'she is the royal queen']

In [3]:
window_size = 2
embedding_size = 5

In [27]:
def __is_bounded(direction, range, index, tokens_leng):
    cover = range * direction
    if cover + index < 0 or cover + index >= tokens_leng:
        return True
    else:
        return False

def get_context(tokens, window_size):
    context_pair = []
    for i, token in enumerate(tokens):
        for j in range(1, window_size+1):
            if not __is_bounded(1, j, i, len(tokens)):
                context_pair.append((tokens[i], tokens[i+j]))
            if not __is_bounded(-1, j, i, len(tokens)):
                context_pair.append((tokens[i], tokens[i-j]))
    return context_pair


def __get_word_set(tokens):
    word_set = set()
    for token in tokens:
        word_set.add(token)
    return word_set

In [28]:
context_pair = []
word_set = set()
for sentence in text:
    tokens = sentence.lower().split()
    context_pair += get_context(tokens, window_size)
    tmp_word_set = __get_word_set(tokens)
    for word in tmp_word_set:
        word_set.add(word)

In [31]:
text

['he is the king', 'the king is royal', 'she is the royal queen']

In [29]:
word_set

{'he', 'is', 'king', 'queen', 'royal', 'she', 'the'}

In [30]:
context_pair

[('he', 'is'),
 ('he', 'the'),
 ('is', 'the'),
 ('is', 'he'),
 ('is', 'king'),
 ('the', 'king'),
 ('the', 'is'),
 ('the', 'he'),
 ('king', 'the'),
 ('king', 'is'),
 ('the', 'king'),
 ('the', 'is'),
 ('king', 'is'),
 ('king', 'the'),
 ('king', 'royal'),
 ('is', 'royal'),
 ('is', 'king'),
 ('is', 'the'),
 ('royal', 'is'),
 ('royal', 'king'),
 ('she', 'is'),
 ('she', 'the'),
 ('is', 'the'),
 ('is', 'she'),
 ('is', 'royal'),
 ('the', 'royal'),
 ('the', 'is'),
 ('the', 'queen'),
 ('the', 'she'),
 ('royal', 'queen'),
 ('royal', 'the'),
 ('royal', 'is'),
 ('queen', 'royal'),
 ('queen', 'the')]

In [33]:
def __get_word_index(word_set):
    word_index_dic = dict()
    inverse_word_dic = dict()
    for i, word in enumerate(word_set):
        word_index_dic[word] = i
        inverse_word_dic[i] = word
    return word_index_dic, inverse_word_dic

In [34]:
word_index_dic, inverse_word_dic = __get_word_index(word_set)
word_size = len(word_set)
batch_size = len(context_pair)

In [35]:
word_index_dic

{'king': 0, 'is': 1, 'she': 2, 'the': 3, 'he': 4, 'queen': 5, 'royal': 6}

In [36]:
inverse_word_dic

{0: 'king', 1: 'is', 2: 'she', 3: 'the', 4: 'he', 5: 'queen', 6: 'royal'}

In [37]:
batch_size

34

In [38]:
word_size

7

In [39]:
inputs = [word_index_dic[x[0]] for x in context_pair]
labels = [word_index_dic[x[1]] for x in context_pair]

In [40]:
inputs

[4,
 4,
 1,
 1,
 1,
 3,
 3,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 1,
 1,
 1,
 6,
 6,
 2,
 2,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 5,
 5]

In [41]:
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, ])

In [43]:
embeddings = tf.Variable(tf.random_uniform([word_size, embedding_size], -1.0, -1.0))

In [44]:
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [47]:
nce_weights = tf.Variable(tf.truncated_normal([word_size, embedding_size], stddev=1.0/math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([word_size]))

In [49]:
prediction = tf.add(tf.matmul(embed, tf.transpose(nce_weights)), nce_biases)

In [50]:
train_labels_vector = tf.one_hot(train_labels, word_size)

In [51]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=train_labels_vector))

In [52]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

In [59]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for iteration in range(10000):
    total_loss = 0

    feed_dict = {train_inputs: inputs, train_labels: labels}
    _, cur_loss, pred = sess.run([optimizer, loss, prediction], feed_dict=feed_dict)
    print('{}: loss: {}'.format(iteration, cur_loss))


0: loss: 1.9196467399597168
1: loss: 1.7732106447219849
2: loss: 1.7572031021118164
3: loss: 1.7466468811035156
4: loss: 1.7375222444534302
5: loss: 1.7288062572479248
6: loss: 1.7200652360916138
7: loss: 1.7110892534255981
8: loss: 1.7017781734466553
9: loss: 1.6920934915542603
10: loss: 1.6820369958877563
11: loss: 1.6716371774673462
12: loss: 1.6609398126602173
13: loss: 1.6500028371810913
14: loss: 1.6388887166976929
15: loss: 1.627661108970642
16: loss: 1.616379737854004
17: loss: 1.6051015853881836
18: loss: 1.5938774347305298
19: loss: 1.5827549695968628
20: loss: 1.5717791318893433
21: loss: 1.5609922409057617
22: loss: 1.5504345893859863
23: loss: 1.5401452779769897
24: loss: 1.5301599502563477
25: loss: 1.5205105543136597
26: loss: 1.5112230777740479
27: loss: 1.502317190170288
28: loss: 1.4938054084777832
29: loss: 1.4856928586959839
30: loss: 1.4779773950576782
31: loss: 1.4706512689590454
32: loss: 1.463701844215393
33: loss: 1.4571126699447632
34: loss: 1.4508659839630127

432: loss: 1.3218287229537964
433: loss: 1.321824312210083
434: loss: 1.32181978225708
435: loss: 1.3218154907226562
436: loss: 1.3218109607696533
437: loss: 1.321806788444519
438: loss: 1.3218024969100952
439: loss: 1.3217980861663818
440: loss: 1.3217939138412476
441: loss: 1.3217896223068237
442: loss: 1.3217854499816895
443: loss: 1.3217812776565552
444: loss: 1.3217769861221313
445: loss: 1.321772813796997
446: loss: 1.3217687606811523
447: loss: 1.3217647075653076
448: loss: 1.3217605352401733
449: loss: 1.3217564821243286
450: loss: 1.3217524290084839
451: loss: 1.3217484951019287
452: loss: 1.321744441986084
453: loss: 1.3217405080795288
454: loss: 1.321736454963684
455: loss: 1.3217326402664185
456: loss: 1.3217287063598633
457: loss: 1.3217248916625977
458: loss: 1.321721076965332
459: loss: 1.3217171430587769
460: loss: 1.3217134475708008
461: loss: 1.3217095136642456
462: loss: 1.3217058181762695
463: loss: 1.321702003479004
464: loss: 1.3216981887817383
465: loss: 1.321694

829: loss: 1.3210159540176392
830: loss: 1.3210151195526123
831: loss: 1.321014165878296
832: loss: 1.321013331413269
833: loss: 1.3210123777389526
834: loss: 1.3210115432739258
835: loss: 1.321010708808899
836: loss: 1.321009635925293
837: loss: 1.3210088014602661
838: loss: 1.3210079669952393
839: loss: 1.321007251739502
840: loss: 1.321006178855896
841: loss: 1.3210054636001587
842: loss: 1.3210045099258423
843: loss: 1.321003794670105
844: loss: 1.321002721786499
845: loss: 1.3210020065307617
846: loss: 1.3210011720657349
847: loss: 1.3210004568099976
848: loss: 1.3209996223449707
849: loss: 1.3209986686706543
850: loss: 1.3209980726242065
851: loss: 1.3209972381591797
852: loss: 1.3209962844848633
853: loss: 1.320995807647705
854: loss: 1.3209949731826782
855: loss: 1.3209943771362305
856: loss: 1.3209935426712036
857: loss: 1.3209928274154663
858: loss: 1.320992350578308
859: loss: 1.3209917545318604
860: loss: 1.3209909200668335
861: loss: 1.3209904432296753
862: loss: 1.3209898

1237: loss: 1.321377158164978
1238: loss: 1.3213893175125122
1239: loss: 1.3213876485824585
1240: loss: 1.32140052318573
1241: loss: 1.3213986158370972
1242: loss: 1.3214120864868164
1243: loss: 1.3214101791381836
1244: loss: 1.321423888206482
1245: loss: 1.3214218616485596
1246: loss: 1.3214361667633057
1247: loss: 1.3214341402053833
1248: loss: 1.3214490413665771
1249: loss: 1.3214468955993652
1250: loss: 1.3214622735977173
1251: loss: 1.321460247039795
1252: loss: 1.3214759826660156
1253: loss: 1.3214739561080933
1254: loss: 1.3214901685714722
1255: loss: 1.3214881420135498
1256: loss: 1.3215047121047974
1257: loss: 1.3215025663375854
1258: loss: 1.3215198516845703
1259: loss: 1.3215175867080688
1260: loss: 1.3215354681015015
1261: loss: 1.321532964706421
1262: loss: 1.3215514421463013
1263: loss: 1.3215489387512207
1264: loss: 1.3215678930282593
1265: loss: 1.3215652704238892
1266: loss: 1.3215850591659546
1267: loss: 1.3215820789337158
1268: loss: 1.3216023445129395
1269: loss: 1.

1653: loss: 1.3214139938354492
1654: loss: 1.3214256763458252
1655: loss: 1.321415901184082
1656: loss: 1.321427822113037
1657: loss: 1.3214181661605835
1658: loss: 1.321429967880249
1659: loss: 1.3214203119277954
1660: loss: 1.3214325904846191
1661: loss: 1.3214226961135864
1662: loss: 1.3214349746704102
1663: loss: 1.321425199508667
1664: loss: 1.3214373588562012
1665: loss: 1.3214274644851685
1666: loss: 1.3214401006698608
1667: loss: 1.3214302062988281
1668: loss: 1.3214426040649414
1669: loss: 1.3214328289031982
1670: loss: 1.3214455842971802
1671: loss: 1.321435570716858
1672: loss: 1.3214482069015503
1673: loss: 1.3214384317398071
1674: loss: 1.3214513063430786
1675: loss: 1.3214411735534668
1676: loss: 1.3214540481567383
1677: loss: 1.321444034576416
1678: loss: 1.3214572668075562
1679: loss: 1.3214471340179443
1680: loss: 1.321460247039795
1681: loss: 1.321450114250183
1682: loss: 1.3214634656906128
1683: loss: 1.3214530944824219
1684: loss: 1.3214665651321411
1685: loss: 1.32

...

8110: loss: 1.3207308053970337
8111: loss: 1.3207271099090576
8112: loss: 1.3207308053970337
8113: loss: 1.320726990699768
8114: loss: 1.3207308053970337
8115: loss: 1.320726990699768
8116: loss: 1.3207306861877441
8117: loss: 1.320726990699768
8118: loss: 1.3207306861877441
8119: loss: 1.320726752281189
8120: loss: 1.3207306861877441
8121: loss: 1.320726752281189
8122: loss: 1.3207305669784546
8123: loss: 1.320726752281189
8124: loss: 1.3207305669784546
8125: loss: 1.320726752281189
8126: loss: 1.320730447769165
8127: loss: 1.3207266330718994
8128: loss: 1.3207303285598755
8129: loss: 1.3207265138626099
8130: loss: 1.3207303285598755
8131: loss: 1.3207265138626099
8132: loss: 1.3207303285598755
8133: loss: 1.3207263946533203
8134: loss: 1.320730209350586
8135: loss: 1.3207263946533203
8136: loss: 1.320730209350586
8137: loss: 1.3207261562347412
8138: loss: 1.3207300901412964
8139: loss: 1.3207261562347412
8140: loss: 1.3207300901412964
8141: loss: 1.3207261562347412
8142: loss: 1.3207

8527: loss: 1.3207145929336548
8528: loss: 1.3207181692123413
8529: loss: 1.3207145929336548
8530: loss: 1.3207181692123413
8531: loss: 1.3207145929336548
8532: loss: 1.3207181692123413
8533: loss: 1.3207143545150757
8534: loss: 1.3207179307937622
8535: loss: 1.3207143545150757
8536: loss: 1.3207180500030518
8537: loss: 1.3207142353057861
8538: loss: 1.3207179307937622
8539: loss: 1.3207142353057861
8540: loss: 1.3207178115844727
8541: loss: 1.3207141160964966
8542: loss: 1.3207178115844727
8543: loss: 1.3207141160964966
8544: loss: 1.3207178115844727
8545: loss: 1.3207141160964966
8546: loss: 1.3207178115844727
8547: loss: 1.320713996887207
8548: loss: 1.3207178115844727
8549: loss: 1.3207138776779175
8550: loss: 1.3207175731658936
8551: loss: 1.3207138776779175
8552: loss: 1.320717453956604
8553: loss: 1.3207138776779175
8554: loss: 1.320717453956604
8555: loss: 1.320713758468628
8556: loss: 1.320717453956604
8557: loss: 1.320713758468628
8558: loss: 1.3207173347473145
8559: loss: 1.

8941: loss: 1.3207032680511475
8942: loss: 1.3207067251205444
8943: loss: 1.3207032680511475
8944: loss: 1.3207067251205444
8945: loss: 1.320703148841858
8946: loss: 1.3207067251205444
8947: loss: 1.320703148841858
8948: loss: 1.3207066059112549
8949: loss: 1.3207030296325684
8950: loss: 1.3207064867019653
8951: loss: 1.3207030296325684
8952: loss: 1.3207063674926758
8953: loss: 1.3207029104232788
8954: loss: 1.3207063674926758
8955: loss: 1.3207027912139893
8956: loss: 1.3207063674926758
8957: loss: 1.3207027912139893
8958: loss: 1.3207063674926758
8959: loss: 1.3207026720046997
8960: loss: 1.3207063674926758
8961: loss: 1.3207026720046997
8962: loss: 1.3207062482833862
8963: loss: 1.3207025527954102
8964: loss: 1.3207061290740967
8965: loss: 1.3207025527954102
8966: loss: 1.3207061290740967
8967: loss: 1.3207025527954102
8968: loss: 1.3207061290740967
8969: loss: 1.3207025527954102
8970: loss: 1.3207060098648071
8971: loss: 1.3207025527954102
8972: loss: 1.3207060098648071
8973: loss

9361: loss: 1.320692777633667
9362: loss: 1.320696234703064
9363: loss: 1.3206926584243774
9364: loss: 1.3206959962844849
9365: loss: 1.3206926584243774
9366: loss: 1.3206959962844849
9367: loss: 1.320692539215088
9368: loss: 1.3206958770751953
9369: loss: 1.3206924200057983
9370: loss: 1.3206959962844849
9371: loss: 1.3206924200057983
9372: loss: 1.3206957578659058
9373: loss: 1.3206924200057983
9374: loss: 1.3206958770751953
9375: loss: 1.3206924200057983
9376: loss: 1.3206957578659058
9377: loss: 1.3206921815872192
9378: loss: 1.3206957578659058
9379: loss: 1.3206923007965088
9380: loss: 1.3206956386566162
9381: loss: 1.3206921815872192
9382: loss: 1.3206955194473267
9383: loss: 1.3206921815872192
9384: loss: 1.3206955194473267
9385: loss: 1.3206921815872192
9386: loss: 1.3206955194473267
9387: loss: 1.3206921815872192
9388: loss: 1.3206952810287476
9389: loss: 1.3206920623779297
9390: loss: 1.3206952810287476
9391: loss: 1.3206920623779297
9392: loss: 1.320695161819458
9393: loss: 

9775: loss: 1.3206830024719238
9776: loss: 1.3206862211227417
9777: loss: 1.3206828832626343
9778: loss: 1.3206861019134521
9779: loss: 1.3206827640533447
9780: loss: 1.3206861019134521
9781: loss: 1.3206828832626343
9782: loss: 1.3206861019134521
9783: loss: 1.3206827640533447
9784: loss: 1.3206861019134521
9785: loss: 1.3206827640533447
9786: loss: 1.3206859827041626
9787: loss: 1.3206827640533447
9788: loss: 1.3206859827041626
9789: loss: 1.3206827640533447
9790: loss: 1.3206859827041626
9791: loss: 1.3206827640533447
9792: loss: 1.320685863494873
9793: loss: 1.3206827640533447
9794: loss: 1.320685863494873
9795: loss: 1.3206825256347656
9796: loss: 1.320685863494873
9797: loss: 1.3206826448440552
9798: loss: 1.320685625076294
9799: loss: 1.3206825256347656
9800: loss: 1.3206857442855835
9801: loss: 1.320682406425476
9802: loss: 1.320685625076294
9803: loss: 1.320682406425476
9804: loss: 1.320685625076294
9805: loss: 1.3206822872161865
9806: loss: 1.3206855058670044
9807: loss: 1.32

In [70]:
def get_vec(word, session):
    return session.run(embeddings[word_index_dic[word]])

def __sim(vec1, vec2):
    return (1 - math.acos(get_cos_similarity(vec1,vec2)) / math.pi)

def get_cos_similarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

def find_cloest_word(word_set, session, target_word):
    sim = 0.0
    vec1 = get_vec(target_word, session)
    result = ''
    for word in word_set:
        if word == target_word:
            continue
        vec2 = get_vec(word,session)
        tmp_sim = __sim(vec1, vec2)
        print('{} : {} : {}'.format(target_word, word, tmp_sim))
        if tmp_sim > sim:
            sim = tmp_sim
            result = word
    return result

In [71]:
find_cloest_word(word_set, sess, 'king')

king : is : 0.4993977721677135
king : she : 0.7711273152027727
king : the : 0.49085532737979043
king : he : 0.7711273152027727
king : queen : 0.779433053181117
king : royal : 0.562655044120014


'queen'